<a href="https://colab.research.google.com/github/MarioROT/IHLT-MAI/blob/main/Session8_MarioRosas_AlamLopez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab session 8 (Syntactic Parser) - IHLT

**Students:**
- Mario Rosas
- Alam Lopez

**Lab Professor:** Salvador Medina Herrera

## Paraphrases Template

In [ ]:
%%shell
git clone https://github.com/mariorot/IHLT-MAI.git
cd 'IHLT-MAI'
mv 'complementary_material' /content/
mv scripts /content/


Cloning into 'IHLT-MAI'...
remote: Enumerating objects: 393, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (150/150), done.
remote: Total 393 (delta 139), reused 89 (delta 53), pack-reused 190
Receiving objects: 100% (393/393), 327.88 KiB | 4.82 MiB/s, done.
Resolving deltas: 100% (229/229), done.


In [ ]:
%load_ext autoreload
%autoreload 2

In [50]:
from scripts.text_preprocessing import TextPreprocessing
from complementary_material.textserver import TextServer
import nltk
from nltk import CFG, ChartParser
!pip install svgling
import svgling


## TODO

- Consider the following sentence:
Lazy cats play with mice.

- Expand the grammar of the example related to non-probabilistic chart parsers in order to subsume this new sentence.

- Perform the constituency parsing using a BottomUpChartParser, a BottomUpLeftCornerChartParser and a LeftCornerChartParser.

- For each one of them, provide the resulting tree, the number of edges and the list of explored edges.

- Which parser is the most efficient for parsing the sentence?

- Which edges are filtered out by each parser and why?

In [48]:
sentence="Lazy cats play with mice."
test_sentence="Lazy cats and mice."

In [49]:
clean_test_sentence=TextPreprocessing.clean_sentence(sentence=test_sentence, lowercase=True, stopwords=False, minwords_len=False,signs=True)

TypeError: ignored

In [ ]:
grammar = CFG.fromstring('''
  NP  -> NNS | JJ NNS | NP CC NP
  NNS -> "cats" | "dogs" | "mice" | NNS CC NNS
  JJ  -> "big" | "small" | "lazy"
  CC  -> "and" | "or"
  ''') #added "lazy" inside the JJ to test

  #Do we need to first pass everything to chomsky normal form?


parser = ChartParser(grammar, trace=1)
parse = parser.chart_parse(clean_test_sentence)




|.  small  .   cats  .   and   .   mice  .|
|[---------]         .         .         .| [0:1] 'small'
|.         [---------]         .         .| [1:2] 'cats'
|.         .         [---------]         .| [2:3] 'and'
|.         .         .         [---------]| [3:4] 'mice'
|[---------]         .         .         .| [0:1] JJ -> 'small' *
|[--------->         .         .         .| [0:1] NP -> JJ * NNS
|.         [---------]         .         .| [1:2] NNS -> 'cats' *
|.         [---------]         .         .| [1:2] NP -> NNS *
|.         [--------->         .         .| [1:2] NNS -> NNS * CC NNS
|[-------------------]         .         .| [0:2] NP -> JJ NNS *
|[------------------->         .         .| [0:2] NP -> NP * CC NP
|.         [--------->         .         .| [1:2] NP -> NP * CC NP
|.         .         [---------]         .| [2:3] CC -> 'and' *
|.         [------------------->         .| [1:3] NNS -> NNS CC * NNS
|[----------------------------->         .| [0:3] NP -> NP CC * NP

In [45]:
print(f'Total edges: {parse.num_edges()}\n')
parse.edges()
#Do we need to add the main sentence?  S -> 'small cats and mice'

Total edges: 28



[[Edge: [0:1] 'small'],
 [Edge: [1:2] 'cats'],
 [Edge: [2:3] 'and'],
 [Edge: [3:4] 'mice'],
 [Edge: [0:1] JJ -> 'small' *],
 [Edge: [0:1] NP -> JJ * NNS],
 [Edge: [1:2] NNS -> 'cats' *],
 [Edge: [1:2] NP -> NNS *],
 [Edge: [1:2] NNS -> NNS * CC NNS],
 [Edge: [0:2] NP -> JJ NNS *],
 [Edge: [0:2] NP -> NP * CC NP],
 [Edge: [1:2] NP -> NP * CC NP],
 [Edge: [2:3] CC -> 'and' *],
 [Edge: [1:3] NNS -> NNS CC * NNS],
 [Edge: [0:3] NP -> NP CC * NP],
 [Edge: [1:3] NP -> NP CC * NP],
 [Edge: [3:4] NNS -> 'mice' *],
 [Edge: [3:4] NP -> NNS *],
 [Edge: [3:4] NNS -> NNS * CC NNS],
 [Edge: [1:4] NNS -> NNS CC NNS *],
 [Edge: [1:4] NP -> NNS *],
 [Edge: [1:4] NNS -> NNS * CC NNS],
 [Edge: [0:4] NP -> JJ NNS *],
 [Edge: [0:4] NP -> NP * CC NP],
 [Edge: [1:4] NP -> NP * CC NP],
 [Edge: [3:4] NP -> NP * CC NP],
 [Edge: [0:4] NP -> NP CC NP *],
 [Edge: [1:4] NP -> NP CC NP *]]

In [46]:
ts = list(parse)
'{num} trees.'.format(num=len(ts))

'28 trees.'

In [40]:
print(ts[0])

[0:1] 'small'


In [43]:
ts[0]

[Edge: [0:1] 'small']

In [42]:
ts[1]

[Edge: [1:2] 'cats']

In [ ]:
from nltk import TopDownChartParser
parser = nltk.TopDownChartParser(grammar)
parse = parser.parse(sent)